# import libraries

In [2]:
import numpy as np
import ipyvolume as ipv

In [7]:
from compas.geometry import Point, Frame, Box
from compas_vol.primitives import VolBox
from compas_vol.microstructures import TPMS
from compas_vol.modifications import Overlay

# create volumetric objects

In [11]:
box = Box(Frame(Point(2,3,1), (1,0.2,-0.3), (0,1,0.4)), 25, 20, 15)
vbox = VolBox(box, 4.0)
gyr = TPMS(0,1.5)
ov = Overlay(vbox, gyr, 0.25)

## sample at discrete intervall (dense grid)

In [12]:
x, y, z = np.ogrid[-20:20:128j,-20:20:128j,-20:20:128j]
dm = ov.get_distance_numpy(x, y, z)

## visualize isosurface

In [14]:
fig = ipv.figure(width=800, height=450)
ipv.xyzlim(20)
mesh = ipv.plot_isosurface(dm, 0.0, color='white', extent=[(-20,20) for _ in range(3)])
ipv.style.use('minimal')
ipv.show()

# apply low pass filter using fourier transform

In [25]:
dft = np.fft.fftn(dm)
dfts = np.fft.fftshift(dft)
lowpass = np.zeros_like(dft,dtype=np.complex128)
fw = 12
a = (128-fw)//2
b = a+fw
lowpass[a:b,a:b,a:b] = dfts[a:b,a:b,a:b]
lowpass = np.fft.fftshift(lowpass)
idft = np.fft.ifftn(lowpass)

## visualize smoothed version

In [28]:
fig = ipv.figure(width=800, height=450)
ipv.xyzlim(20)
mesh = ipv.plot_isosurface(idft.real, 0.0, color='pink', extent=[(-20,20) for _ in range(3)])
ipv.style.use('minimal')
ipv.show()

/Users/bernham/anaconda3/envs/compasdev/lib/python3.9/site-packages/ipyvolume/pylab.py:742: FutureWarning: marching_cubes_lewiner is deprecated in favor of marching_cubes. marching_cubes_lewiner will be removed in version 0.19
  values = skimage.measure.marching_cubes_lewiner(data, level)


In [24]:
a = np.zeros((7,7))
a[2:5,2:5] = 1
a

array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 0., 0.],
       [0., 0., 1., 1., 1., 0., 0.],
       [0., 0., 1., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]])